In [1]:
import pandas as pd
import json
import sqlite3


In [2]:
orders = pd.read_csv('orders.csv')
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
with open('users.json') as f:
    users = pd.DataFrame(json.load(f))

users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(':memory:')

with open('restaurants.sql') as f:
    conn.executescript(f.read())

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
df = orders.merge(users, on='user_id', how='left') \
           .merge(restaurants, on='restaurant_id', how='left')

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
df.shape


(10000, 12)

In [13]:
df[df.membership == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [12]:
df[df.membership == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [14]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
df['quarter'] = df['order_date'].dt.quarter

df.groupby('quarter')['total_amount'].sum()


,total_amount
quarter,
1,2010626.64
2,1945348.72
3,2037385.10
4,2018263.66


In [15]:
df[df.membership == 'Gold'] \
  .groupby('city')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [16]:
df.groupby('cuisine')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [17]:
df.groupby('user_id')['total_amount'] \
  .sum() \
  .loc[lambda x: x > 1000] \
  .count()


np.int64(2544)

In [18]:
def rating_bucket(r):
    if 3.0 <= r <= 3.5: return '3.0–3.5'
    if 3.6 <= r <= 4.0: return '3.6–4.0'
    if 4.1 <= r <= 4.5: return '4.1–4.5'
    if 4.6 <= r <= 5.0: return '4.6–5.0'

df['rating_bucket'] = df['rating'].apply(rating_bucket)

df.groupby('rating_bucket')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


,total_amount
rating_bucket,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [19]:
df[df.membership == 'Gold'] \
  .groupby('city')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [20]:
pd.concat([
    df.groupby('cuisine')['restaurant_id'].nunique(),
    df.groupby('cuisine')['total_amount'].sum()
], axis=1)


,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [21]:
round((df[df.membership == 'Gold'].shape[0] / df.shape[0]) * 100)


50

In [22]:
df.groupby('restaurant_name_y').agg(
    orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).query('orders < 20') \
 .sort_values('avg_order_value', ascending=False)


,orders,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [23]:
df.groupby(['membership', 'cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [24]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
df['quarter'] = df['order_date'].dt.quarter

df.groupby('quarter')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [25]:
df[df.membership == 'Gold'].shape[0]


4987

In [26]:
round(df[df.city == 'Hyderabad']['total_amount'].sum())


1889367

In [27]:
df['user_id'].nunique()


2883

In [28]:
round(df[df.membership == 'Gold']['total_amount'].mean(), 2)


np.float64(797.15)

In [29]:
df[df.rating >= 4.5].shape[0]


3374

In [30]:
top_city = df[df.membership == 'Gold'] \
             .groupby('city')['total_amount'] \
             .sum() \
             .idxmax()

df[(df.membership == 'Gold') & (df.city == top_city)].shape[0]


1337